In [1]:
import pandas as pd
import numpy as np

In [1]:
def prefilter_items(data, item_features=None, take_n_popular=5000):
    # Уберем самые популярные товары
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id':'share_unique_users'}, inplace=True)
    pop_item = popularity[popularity.share_unique_users>0.2].item_id.tolist()

    data = data[~data.isin(pop_item)]
    
    # Уберем самые НЕ популярные товары
    not_pop_item = popularity[popularity.share_unique_users<0.2].item_id.tolist()
    data = data[~data.isin(not_pop_item)]
    
    # Уберем товары, которые не продавались за последние N месяцев
    month_count = 12
    not_buy_days_filter = int(data.day.max() - month_count*30)
    data[data.day>=not_buy_days_filter]
    
    # Уберем не интересные для рекоммендаций категории
    if item_features is not None:
        category_count_items = item_features.groupby('department').item_id.nunique().reset_index(name='count_item')
        not_pop_cat = category_count_items[category_count_items.count_item<150].department.to_list()
        not_pop_cat_items = item_features[item_features.department.isin(not_pop_cat)].item_id.to_list()
        data[~data.item_id.isin(not_pop_cat_items)]
    
    # Уберем слишком дешевые товары
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    min_price = 2
    data = data[data.price>min_price]
    
    # Уберем слишком дорогие товары
    max_price = 50
    data = data[data.price<max_price]
    
    #Возьмем только топ N популярных товаров
    popularity = data.groupby('item_id').quantity.sum().reset_index()
    top_popular = popularity.sort_values(by='quantity', ascending=False).head(take_n_popular).item_id.to_list()
    #заведем фиктивный товар, которым заменим все остальные товары не из топ-N
    data.loc[~data['item_id'].isin(top_popular), 'item_id'] = 999999
    
    return data